In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import ssl
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import sys
sys.path.append('../')
from fun.models import *
sns.set_style("whitegrid")
from torch.utils.data import random_split
from tqdm import tqdm
import pandas as pd
torch.manual_seed(42)
np.random.seed(42)



In [ ]:
ssl._create_default_https_context = ssl._create_unverified_context


### Prepare dataset and split into valid-train set

In [5]:


transform = transforms.Compose([
    transforms.ToTensor()
])

train_ds = torchvision.datasets.CIFAR10(
    root="../data",
    train=True,
    download=True,
    transform=transform

)

test_ds = torchvision.datasets.CIFAR10(
    root="../data",
    train=False,
    download=True,
    transform=transform
)


    
    

Files already downloaded and verified
Files already downloaded and verified


In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

def train(dataloader, model, loss_fn, optimizer):

    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.train()
    train_loss, correct = 0, 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    train_loss /= num_batches
    correct /= size
    print(f" Train Accuracy: {(100*correct):>0.1f}%, Train Avg loss {train_loss:>8f} \n")
    return correct, train_loss


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size

    print(f"Test Accuracy: {(100*correct):>0.1f}%, Test Avg loss: {test_loss:>8f} \n")
    return correct, test_loss
    

Using cuda device


## Learning Rate

In [ ]:
#------------------------------------------------------------------
#--------------------- DO PUSZCZENIA ------------------------------
#------------------------------------------------------------------
learning_rates = [1e-6, 1e-5,1e-4,1e-3,0.01, 0.1]
batch_size = 4
epochs = 25
data = np.array([[0]*len(learning_rates)]*epochs)
lr_train_acc_history = pd.DataFrame(data, columns = learning_rates)
lr_test_acc_history = pd.DataFrame(data, columns = learning_rates)
lr_train_loss_history  = pd.DataFrame(data, columns = learning_rates)
lr_test_loss_history  = pd.DataFrame(data, columns = learning_rates)

train_dataloader = DataLoader(
        train_ds,
        batch_size=batch_size, 
        shuffle=True
        )
test_dataloader = DataLoader(
    valid_ds, 
    batch_size=batch_size,
    shuffle=False
    )

net = Net().to(device)
criterion = nn.CrossEntropyLoss()

for lr in learning_rates:
    print(f"Learning Rate: {lr}\n********************************")
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)

    for t in range(epochs):

        print(f"Epoch {t+1}\n-------------------------------")
        train_acc, train_loss = train(train_dataloader, net, criterion, optimizer)
        test_acc, test_loss = test(test_dataloader, net, criterion)
        lr_train_acc_history.loc[t,lr] = train_acc        
        lr_test_acc_history.loc[t,lr] = test_acc
        lr_train_loss_history.loc[t,lr] = train_loss
        lr_test_loss_history.loc[t,lr] = test_loss

lr_train_acc_history.to_csv("../data/results/lr_train_acc_history.csv")
lr_train_acc_history.to_csv("../data/results/lr_train_acc_history.csv")
lr_train_loss_history.to_csv("../data/results/lr_train_loss_history.csv")
lr_test_loss_history.to_csv("../data/results/lr_test_loss_history.csv")

        

## Batch size experiments

In [ ]:
#------------------------------------------------------------------
#--------------------- DO PUSZCZENIA ------------------------------
#------------------------------------------------------------------
epochs = 25
batch_sizes = [2,4,8,16,32]
lr = 1e-3
data = np.array([[0]*len(batch_sizes)]*epochs)
batch_train_acc_history = pd.DataFrame(data, columns = batch_sizes)
batch_test_acc_history = pd.DataFrame(data, columns = batch_sizes)
batch_train_loss_history  = pd.DataFrame(data, columns = batch_sizes)
batch_test_loss_history  = pd.DataFrame(data, columns = batch_sizes)

for batch_size in batch_sizes:
    print(f"Batch size: {batch_size} \n")
    
    train_dataloader = DataLoader(
        train_ds,
        batch_size=batch_size, 
        shuffle=True
        )
    test_dataloader = DataLoader(
        valid_ds, 
        batch_size=batch_size,
        shuffle=False
        )

    net = Net().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    max_accuracy_in_batch=0
    epoch = 1
    for t in range(epochs):

        print(f"Epoch {t+1}\n-------------------------------")
        train_acc, train_loss = train(train_dataloader, net, criterion, optimizer)
        test_acc, test_loss = test(test_dataloader, net, criterion)
        batch_train_acc_history.loc[t,batch_size] = train_acc        
        batch_test_acc_history.loc[t,batch_size] = test_acc
        batch_train_loss_history.loc[t,batch_size] = train_loss
        batch_test_loss_history.loc[t,batch_size] = test_loss

batch_train_acc_history.to_csv("../data/results/batch_train_acc_history.csv")
batch_test_acc_history.to_csv("../data/results/batch_test_acc_history.csv")
batch_train_loss_history.to_csv("../data/results/batch_train_loss_history.csv")
batch_test_loss_history.to_csv("../data/results/batch_test_loss_history.csv")


## Dropout

In [ ]:
#------------------------------------------------------------------
#--------------------- DO PUSZCZENIA ------------------------------
#------------------------------------------------------------------
batch_size = 4
epochs = 20
learning_rate = 1e-3
dropouts = [0.1, 0.2, 0.25, 0.35, 0.5]
data = np.array([[0]*len(dropouts)]*epochs)
dropouts_train_acc_history = pd.DataFrame(data, columns = dropouts)
dropouts_test_acc_history = pd.DataFrame(data, columns = dropouts)
dropouts_train_loss_history  = pd.DataFrame(data, columns = dropouts)
dropouts_test_loss_history  = pd.DataFrame(data, columns = dropouts)
train_dataloader = DataLoader(
        train_ds,
        batch_size=batch_size, 
        shuffle=True
        )
test_dataloader = DataLoader(
    valid_ds, 
    batch_size=batch_size,
    shuffle=False
    )

for drop_out in dropouts:

    print(f"Dropout {drop_out}\n-------------------------------")
    net = Net_dropout(dropout_rate=drop_out).to(device)
    criterion = nn.CrossEntropyLoss()

    optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

    for t in range(epochs):
        
        print(f"Epoch {t+1}\n-------------------------------")
        train_acc, train_loss = train(train_dataloader, net, criterion, optimizer)
        test_acc, test_loss = test(test_dataloader, net, criterion)
        dropouts_train_acc_history.loc[t,drop_out] = train_acc        
        dropouts_train_acc_history.loc[t,drop_out] = test_acc
        dropouts_train_loss_history.loc[t,drop_out] = train_loss
        dropouts_test_loss_history.loc[t,drop_out] = test_loss

dropouts_train_acc_history.to_csv("../data/results/dropouts_train_acc_history.csv")    
dropouts_train_acc_history.to_csv("../data/results/dropouts_train_acc_history.csv")
dropouts_train_loss_history.to_csv("../data/results/dropouts_train_loss_history.csv")
dropouts_test_loss_history.to_csv("../data/results/dropouts_test_loss_history.csv")

## Final with dropout small 

In [ ]:
#------------------------------------------------------------------
#--------------------- DO PUSZCZENIA ------------------------------
#------------------------------------------------------------------
batch_size = 4
epochs = 20
learning_rate = 1e-3
dropout_rate = 0.25

data = np.array([[0]*4]*epochs)
dropouts_train_test_final = pd.DataFrame(data, columns = ["test_acc", "train_acc", "test_loss", "train_loss"])


train_dataloader = DataLoader(
        train_ds,
        batch_size=batch_size, 
        shuffle=True
        )
test_dataloader = DataLoader(
    valid_ds, 
    batch_size=batch_size,
    shuffle=False
    )


net = Net_dropout(dropout_rate=dropout_rate).to(device)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

for t in range(epochs):
    
    print(f"Epoch {t+1}\n-------------------------------")
    train_acc, train_loss = train(train_dataloader, net, criterion, optimizer)
    test_acc, test_loss = test(test_dataloader, net, criterion)
    dropouts_train_test_final.loc[t,"test_acc"] = test_acc
    dropouts_train_test_final.loc[t,"train_acc"] = train_acc
    dropouts_train_test_final.loc[t,"test_loss"] = test_loss
    dropouts_train_test_final.loc[t,"train_loss"] = train_loss

dropouts_train_test_final.to_csv("../data/results/dropouts_train_test_final.csv")
 

## Final with dropdown bigger

In [ ]:
#------------------------------------------------------------------
#--------------------- DO PUSZCZENIA ------------------------------
#------------------------------------------------------------------
batch_size = 4
epochs = 20
learning_rate = 1e-3
dropout_rate = 0.25

data = np.array([[0]*4]*epochs)
dropouts_bigger_train_test_final = pd.DataFrame(data, columns = ["test_acc", "train_acc", "test_loss", "train_loss"])


train_dataloader = DataLoader(
        train_ds,
        batch_size=batch_size, 
        shuffle=True
        )
test_dataloader = DataLoader(
    valid_ds, 
    batch_size=batch_size,
    shuffle=False
    )


net = Net_dropout_new(dropout_rate=dropout_rate).to(device)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

for t in range(epochs):
    
    print(f"Epoch {t+1}\n-------------------------------")
    tr_acc, tr_loss = train(train_dataloader, net, criterion, optimizer)
    test_acc, test_loss = test(test_dataloader, net, criterion)
    dropouts_bigger_train_test_final.loc[t,"test_acc"] = test_acc
    dropouts_bigger_train_test_final.loc[t,"train_acc"] = train_acc
    dropouts_bigger_train_test_final.loc[t,"test_loss"] = test_loss
    dropouts_bigger_train_test_final.loc[t,"train_loss"] = train_loss

dropouts_bigger_train_test_final.to_csv("../data/results/dropouts_bigger_train_test_final.csv")
 

## WEIGHT DECAY - ADAMW

In [7]:
#------------------------------------------------------------------
#--------------------- DO PUSZCZENIA ------------------------------
#------------------------------------------------------------------
epochs = 30
batch_size = 8
weight_decay = [0.001, 0.01, 0.1, 0.25, 0.5]
lr = 1e-3
data = np.array([[0]*len(weight_decay)]*epochs)
wd_train_acc_history = pd.DataFrame(data, columns = weight_decay)
wd_test_acc_history = pd.DataFrame(data, columns = weight_decay)
wd_train_loss_history  = pd.DataFrame(data, columns = weight_decay)
wd_test_loss_history  = pd.DataFrame(data, columns = weight_decay)

train_dataloader = DataLoader(
    train_ds,
    batch_size=batch_size, 
    shuffle=True
    )
test_dataloader = DataLoader(
    test_ds, 
    batch_size=batch_size,
    shuffle=False
    )

for wd in weight_decay:
    print(f"Batch size: {batch_size} \n")    
    

    net = Net().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(net.parameters(), lr=lr, weight_decay=wd)
    
    for t in range(epochs):

        print(f"Epoch {t+1}\n-------------------------------")
        train_acc, train_loss = train(train_dataloader, net, criterion, optimizer)
        test_acc, test_loss = test(test_dataloader, net, criterion)
        wd_train_acc_history.loc[t,wd] = train_acc        
        wd_test_acc_history.loc[t,wd] = test_acc
        wd_train_loss_history.loc[t,wd] = train_loss
        wd_test_loss_history.loc[t,wd] = test_loss

wd_train_acc_history.to_csv("../data/results/bs8_ep30_lr-3_wd/wd_train_acc_history.csv")

wd_test_acc_history.to_csv("../data/results/bs8_ep30_lr-3_wd/wd_test_acc_history.csv")

wd_train_loss_history.to_csv("../data/results/bs8_ep30_lr-3_wd/wd_train_loss_history.csv")

wd_test_loss_history.to_csv("../data/results/bs8_ep30_lr-3_wd/wd_test_loss_history.csv")



Batch size: 8 

Epoch 1
-------------------------------
 Train Accuracy: 50.9%, Train Avg loss 1.366056 

Test Accuracy: 59.0%, Test Avg loss: 1.140588 

Epoch 2
-------------------------------
 Train Accuracy: 62.3%, Train Avg loss 1.068926 

Test Accuracy: 63.5%, Test Avg loss: 1.032446 

Epoch 3
-------------------------------
 Train Accuracy: 66.2%, Train Avg loss 0.960117 

Test Accuracy: 64.4%, Test Avg loss: 1.022269 

Epoch 4
-------------------------------
 Train Accuracy: 68.9%, Train Avg loss 0.888816 

Test Accuracy: 64.9%, Test Avg loss: 1.007903 

Epoch 5
-------------------------------
 Train Accuracy: 70.8%, Train Avg loss 0.833821 

Test Accuracy: 67.2%, Test Avg loss: 0.949934 

Epoch 6
-------------------------------
 Train Accuracy: 72.0%, Train Avg loss 0.792762 

Test Accuracy: 67.4%, Test Avg loss: 0.930169 

Epoch 7
-------------------------------
 Train Accuracy: 73.4%, Train Avg loss 0.753727 

Test Accuracy: 67.9%, Test Avg loss: 0.936816 

Epoch 8
----------